In [1]:
import findspark
findspark.init('/home/kajili/spark-2.4.5-bin-hadoop2.7')

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('Ecommerce_Customers.csv',inferSchema=True, header=True)

In [6]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [7]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [34]:
for person in data.head(5):
    print(person)
    print('\n')

Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005)


Row(Email='hduke@hotmail.com', Address='4547 Archer CommonDiazchester, CA 06566-8576', Avatar='DarkGreen', Avg Session Length=31.92627202636016, Time on App=11.109460728682564, Time on Website=37.268958868297744, Length of Membership=2.66403418213262, Yearly Amount Spent=392.2049334443264)


Row(Email='pallen@yahoo.com', Address='24645 Valerie Unions Suite 582Cobbborough, DC 99414-7564', Avatar='Bisque', Avg Session Length=33.000914755642675, Time on App=11.330278057777512, Time on Website=37.110597442120856, Length of Membership=4.104543202376424, Yearly Amount Spent=487.54750486747207)


Row(Email='riverarebecca@gmail.com', Address='1414 David ThroughwayPort Jason, OH 22070-1220', Avatar='Sadd

In [12]:
for item in data.head(1)[0]:
    print(item)

mstephenson@fernandez.com
835 Frank TunnelWrightmouth, MI 82180-9605
Violet
34.49726772511229
12.65565114916675
39.57766801952616
4.0826206329529615
587.9510539684005


In [36]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [37]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [38]:
assembler = VectorAssembler(
    inputCols=["Avg Session Length", "Time on App", 
               "Time on Website",'Length of Membership'],
    outputCol="features")

In [39]:
output = assembler.transform(data)

In [41]:
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [43]:
output.select('features').show()

+--------------------+
|            features|
+--------------------+
|[34.4972677251122...|
|[31.9262720263601...|
|[33.0009147556426...|
|[34.3055566297555...|
|[33.3306725236463...|
|[33.8710378793419...|
|[32.0215955013870...|
|[32.7391429383803...|
|[33.9877728956856...|
|[31.9365486184489...|
|[33.9925727749537...|
|[33.8793608248049...|
|[29.5324289670579...|
|[33.1903340437226...|
|[32.3879758531538...|
|[30.7377203726281...|
|[32.1253868972878...|
|[32.3388993230671...|
|[32.1878120459321...|
|[32.6178560628234...|
+--------------------+
only showing top 20 rows



In [44]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [45]:
final_data = output.select(['features', 'Yearly Amount Spent'])

In [46]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [47]:
# Split final_data into train_data and test_data
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [49]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                335|
|   mean| 497.68316836961355|
| stddev|  81.30020739776381|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+



In [50]:
test_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                165|
|   mean| 502.62519833621116|
| stddev|  75.25479858605739|
|    min| 308.52774655803336|
|    max|  765.5184619388373|
+-------+-------------------+



In [51]:
# Create LinearRegression Object
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [52]:
# Create LinearRegression Model and then train it using LinearRegression object with train_data as the data.
lr_model = lr.fit(train_data)

In [53]:
# evaluate the model with test_data
test_results = lr_model.evaluate(test_data)

***Residuals*** are just the difference between the predicted value and the actual label from the test data

In [54]:
# Show residuals from test_results.
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
|  7.073003315310643|
|  4.320424150002168|
|-12.719039883810353|
| -21.05898692867811|
| -3.221769188026201|
|  3.381605684720512|
|  4.294834559017431|
| -8.048113815679585|
| -5.505264985194913|
|-17.404613995762702|
|   18.2973963292568|
|  17.62310391958073|
| -4.066289076656005|
| -26.39619098481296|
| -7.609488528808129|
|  -5.79228177120757|
| 2.5030769875536976|
|  1.187202211425074|
|  8.681933431388018|
| -8.997850850786733|
+-------------------+
only showing top 20 rows



### Root Mean Squared Error:
- This is the average error
- The difference between your test value vs the predicted value

In [55]:
test_results.rootMeanSquaredError

9.596067431778767

### R Squared:
- This is the percentage of accuracy of the model (?)
- basically says "My model explains 98% of the variance in the data"

In [56]:
test_results.r2

0.9836409423486154

In [57]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [58]:
unlabeled_data = test_data.select('features')

In [60]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.9716756438877...|
|[31.0472221394875...|
|[31.0662181616375...|
|[31.1239743499119...|
|[31.2681042107507...|
|[31.3091926408918...|
|[31.3584771924370...|
|[31.4474464941278...|
|[31.5257524169682...|
|[31.5702008293202...|
|[31.6005122003032...|
|[31.6098395733896...|
|[31.6253601348306...|
|[31.6739155032749...|
|[31.7207699002873...|
|[31.7242025238451...|
|[31.7366356860502...|
|[31.8293464559211...|
|[31.8512531286083...|
|[31.8854062999117...|
+--------------------+
only showing top 20 rows



In [62]:
predictions = lr_model.transform(unlabeled_data)

In [64]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.9716756438877...| 487.5656064415821|
|[31.0472221394875...| 388.1769750390192|
|[31.0662181616375...| 461.6523330914847|
|[31.1239743499119...| 508.0060407684439|
|[31.2681042107507...| 426.6923023618501|
|[31.3091926408918...| 429.3391121552131|
|[31.3584771924370...|  490.881115890458|
|[31.4474464941278...| 426.6508559109036|
|[31.5257524169682...| 449.4708917950768|
|[31.5702008293202...| 563.3501061371676|
|[31.6005122003032...|460.87545516184014|
|[31.6098395733896...|426.92244573152743|
|[31.6253601348306...| 380.4031898335802|
|[31.6739155032749...|502.12125889469417|
|[31.7207699002873...| 546.3844220068311|
|[31.7242025238451...|509.18016905916807|
|[31.7366356860502...| 494.4303692679782|
|[31.8293464559211...| 383.9651357765499|
|[31.8512531286083...| 464.3103132354104|
|[31.8854062999117...|399.10112382326224|
+--------------------+------------

In [65]:
predictions.describe().show()

+-------+-----------------+
|summary|       prediction|
+-------+-----------------+
|  count|              165|
|   mean|502.9109966567498|
| stddev| 75.1363320742046|
|    min|315.4375204508726|
|    max|766.3457714663111|
+-------+-----------------+

